# 三普耕地质量等级评价

In [10]:
import os
import time
import pandas as pd 
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
# 土类结果图shp路径
soil_type_shp_path = r'F:\cache_data\shp_file\sb\sb_soiltype_result_rf_20241029.shp'

In [12]:
# 筛选类型为耕地的shp
land_list = ["水田","水浇地","旱地"]
all_soil_type_df = gpd.read_file(soil_type_shp_path)
soil_type_df = all_soil_type_df[all_soil_type_df['DLMC'].isin(land_list)]

In [16]:

# 融合DLMC和Class1_tl相同的图斑,不创建多部件
# 先重置索引以避免DLMC的歧义
soil_type_df = soil_type_df.reset_index(drop=True)
soil_type_df = soil_type_df.dissolve(by=['DLMC', 'Class1_tl'], aggfunc={
    'DLBM': 'first',
    'MZMC': 'first'
})

# 将多部件要素拆分为单部件
soil_type_df = soil_type_df.explode(index_parts=True)

# 重置索引，使用drop=False来保留原索引列，并指定新的列名
soil_type_df = soil_type_df.reset_index(level=['DLMC', 'Class1_tl'])

# 添加面积字段，并计算面积
soil_type_df['area'] = soil_type_df.geometry.area

# 保存结果
try:
    output_path = r'C:\Users\Runker\Desktop\test\shp\sb_soiltype_result.shp'
    soil_type_df.to_file(output_path, driver='ESRI Shapefile',encoding='utf-8')
    print(f"文件已保存至: {output_path}")
    
    # 打印结果统计信息
    print("\n处理结果统计：")
    print(f"总要素数量：{len(soil_type_df)}")
    print("\n按地类统计：")
    print(soil_type_df['DLMC'].value_counts())
    print("\n按土类等级统计：")
    print(soil_type_df['Class1_tl'].value_counts())
except Exception as e:
    print(f"保存文件时出错: {str(e)}")

文件已保存至: C:\Users\Runker\Desktop\test\shp\sb_soiltype_result.shp

处理结果统计：
总要素数量：34405

按地类统计：
DLMC
旱地    23564
水田    10841
Name: count, dtype: int64

按土类等级统计：
Class1_tl
黄壤     11366
石灰土    10866
水稻土    10841
潮土       595
紫色土      585
粗骨土      139
黄棕壤       13
Name: count, dtype: int64
